In [ ]:
# A live monitor of the effects of Freping and AFC

DEST_IP = "127.0.0.1"
MY_IP = "127.0.0.1"

# DEST_IP = "ospboard.lan"
# DEST_IP = "192.168.86.38"
# MY_IP = "192.168.86.26"

In [ ]:
# optionally make Jupyter use the whole width of the browser
from IPython.display import Image, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import plotly.graph_objects as go
from IPython.display import Audio
from osp_control import OspControl
import os
import json
import numpy as np
import time
from ipywidgets import Checkbox, FloatSlider, Dropdown, ToggleButton, FloatText, Output, VBox, HBox, Layout
import socket
import threading
import sys

In [ ]:
freqs = { 
    6:  [250, 500, 1000, 2000, 4000, 8000, 0, 0, 0, 0, 0],
    10: [250, 500, 750, 1000, 1500, 2000, 3000, 4000, 6000, 8000, 0],
    11: [250, 354, 500, 707, 1000, 1414, 2000, 2828, 4000, 5657, 8000]}

plot_log = True

def plot_fft(fig, title, out1, update=True):
    ftrans = 10 * np.log10(np.abs(np.fft.fft(out1)/len(out1)))
    outlen = len(out1)
    values = np.arange(int(outlen/2))
    period = outlen/48000
    frequencies = values/period

    if update:
        with fig.batch_update():
            fig.data[0].y = ftrans
            fig.data[0].x = frequencies
            fig.update_layout()
            return
        
   
    fig.add_trace(go.Scatter(x=frequencies, line_color='green', opacity=1, y=ftrans))
    fig.update_layout(title=title,
                    xaxis_title='Frequency',
                    yaxis_title='dbFS',
                    template='plotly_white')

def plot_mono(fig, title, out1, lab1, rate=48000, update=True):

    # Create x values.
    x = np.array(range(len(out1)))/rate
    if update:
        with fig.batch_update():
            fig.data[0].y = out1
            fig.data[0].x = x
            fig.update_layout()
            return
    fig.add_trace(go.Scatter(x=x,
                             y=out1,
                             name=lab1,
                             opacity=1))
    fig.update_layout(title=title,
                      yaxis_range=[-1,1],
                      xaxis_title='Time(sec)',
                      yaxis_title='Amplitude',
                      template='plotly_white')

In [ ]:
# Connect to OSP process
osp = OspControl(DEST_IP)

# set the number of bands and mute mic
# osp.send({"method": "set", "data": {"num_bands": 11}})
osp.send_chan({"alpha": 0})

# get the current number of bands
vals = osp.send({"method": "get"})
cur_bands = vals['num_bands']

In [ ]:
update = False
def monitor(afig, ffig, interval=2):
    global update
    PORT = 5001
    sample_size = 384 * 1000  # 384 bytes per ms. Save 1000ms
    
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_address = (MY_IP, PORT)
    sock.bind(server_address)
    sock.listen(1)
    filename = f"tcp://{MY_IP}:{PORT}"
    osp.send_chan({'audio_rfile': filename, "audio_record": 1})
    data = b''
    done = False
    
    # Wait for a connection
    with out:
        print('waiting for a connection')
    connection, client_address = sock.accept()
    try:
        with out:
            print('connection from', client_address)

        # Receive the data in small chunks and retransmit it
        while True:
            while len(data) < sample_size:
                new_data = connection.recv(32768)
                if len(new_data) == 0:
                    done = True
                    break
                data += new_data
            if done:
                break
            data = np.frombuffer(data, dtype=np.float32)
            data = np.reshape(data, (2, -1), 'F')
            # plot
            plot_mono(afig, '', data[0], 'out', update=update)
            plot_fft(ffig, '',  data[0], update=update)
            if update == False:
                update = True
            data = b''
            new_data = connection.recv(128000) # discard
    finally:
        with out:
            print("Closing socket")
        # Clean up the connection
        connection.close()
        sock.close()

In [ ]:
# Create Controls

freping = Checkbox(
    value=False,
    description='Freping',
    disabled=False,
    indent=True
)

# create 11 sliders. We will only display the ones necessary
freping_alpha = []
for band in range(11):
    freping_alpha.append(FloatSlider(
        value=0,
        min=-.5,
        max=.5,
        step=0.01,
        description=f"{freqs[cur_bands][band]}",
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='.2f'
    ))

afc = Checkbox(
    description='AFC',
)

afc_type = Dropdown(
    options=[('FXLMS', 1), ('IPNLMS', 2), ('SLMS', 3)],
    value=1,
    description='Type:',
)

play_w = Dropdown(
    options=[('diner', 'Freping/MPEG_es01_s.wav'), ('animals', 'Freping/MPEG_es03_s.wav'), ('sound', 'Freping/MPEG_te19.wav')],
    value='Freping/MPEG_es01_s.wav',
    description='Play:',
)

play_b = ToggleButton(
    description='Play',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Play File',
    icon='play'
)

band_w = Dropdown(
    options=[6, 10, 11],
    value=cur_bands,
    description='Bands:',
)
alpha_w = ToggleButton(
    value=False,
    description='File Audio',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='File input',
    icon='file'
)
g65_w = FloatText(
    value=0,
    description='G65:'
)
cr_w =  FloatText(
    value=1.0,
    description='CR:'
)
freping_reset = ToggleButton(
    description='Reset Freping',
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Reset all values to 0',
    icon='recycle'
)

In [ ]:
def compute_g65(vals):
    vals = vals['left']
    g50 = np.array(vals['g50'])
    g80 = np.array(vals['g80'])
    slope = (g80 - g50) / 30
    g65 = g50 + slope * 15
    cr = 30/((80 + g80) - (50 + g50))
    return g65[0], cr[0]

def compute_g50(g65, cr):
    slope = (1-cr) / cr
    g50 = g65 - slope * 15
    g80 = g65 + slope * 15
    return g50, g80

In [ ]:
# called when controls change
def update_osp(k=None):
    w = k['owner']
    if w == freping:
        osp.send_chan({"freping": w.value})
        return
    if w == afc:
        osp.send_chan({"afc": w.value})
        return
    if w == afc_type:
        osp.send_chan({"afc_type": w.value})
        return
    if w == g65_w or w == cr_w:
        g50, g80 = compute_g50(g65_w.value, cr_w.value)
        osp.send_chan({"g50": g50, "g80": g80})
        return
    if w == alpha_w:
        if alpha_w.value:
            osp.send_chan({"alpha": 0})
            alpha_w.description = 'Live'
            alpha_w.button_style='success'
            alpha_w.tooltip='Live (mic) input'
            alpha_w.icon='microphone'
            return
        osp.send_chan({"alpha": 1})
        alpha_w.description = 'File Audio'
        alpha_w.button_style='info'
        alpha_w.tooltip='File input'
        alpha_w.icon='file'
        return
    fvec = [freping_alpha[x].value for x in range(cur_bands)]
    osp.send_chan({"freping_alpha": fvec})

In [ ]:
def band_cb(k):
    global cur_bands, fsliders
    cur_bands = band_w.value
    osp.send({"method": "set", "data": {"num_bands": cur_bands}})
    vals = osp.send({"method": "get"})
    for x in range(cur_bands):
        freping_alpha[x].description = f"{freqs[cur_bands][x]}"
        freping_alpha[x].value = vals['left']['freping_alpha'][x]
    fsliders.children = [freping_alpha[x] for x in range(cur_bands)]
    alpha_w.value = False if vals['left']['alpha'] == 1.0 else True
    freping.value = vals['left']['freping']
    afc.value = vals['left']['afc']
    afc_type.value = vals['left']['afc_type']
    g65, cr = compute_g65(vals)
    g65_w.value = g65
    cr_w.value = cr

def freset_cb(w):
    for band in range(11):
        freping_alpha[band].value = 0

In [ ]:
def play_cb(k):
    osp.send_chan({"audio_filename": play_w.value, "audio_play": 1, "audio_alpha": 1}, 'left')

play_b.observe(play_cb, names='value')

In [ ]:
# set callbacks
play_b.observe(play_cb, names='value')
alpha_w.observe(update_osp, names='value')
g65_w.observe(update_osp, names='value')
cr_w.observe(update_osp, names='value')
freping.observe(update_osp, names='value')
afc.observe(update_osp, names='value')
afc_type.observe(update_osp, names='value')
for band in range(11):
    freping_alpha[band].observe(update_osp, names='value')
band_w.observe(band_cb, names='value')
freping_reset.observe(freset_cb, names='value')

In [ ]:
start_b = ToggleButton(
    value=False,
    description='Start',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Start',
    icon='play'
)
def start_cb(k):
    if start_b.value:
        out.clear_output()
        start_b.description = 'Stop'
        start_b.button_style='danger'
        start_b.tooltip='Stop Analysis'
        start_b.icon='stop'
        threading.Thread(target=monitor, args=(audio_fig, fft_fig)).start()
        return
    osp.send_chan({"audio_record": 0})
    start_b.description = 'Start'
    start_b.button_style='success'
    start_b.tooltip='Start'
    start_b.icon='play'
    
start_b.observe(start_cb, names='value')  

In [ ]:
# build GUI
out = Output(layout={'border': '1px solid black'})
audio_fig = go.FigureWidget()
fft_fig = go.FigureWidget()
wdrc_box = HBox([g65_w, cr_w], layout={'border': 'solid 1px'})
play_box = HBox([play_w, play_b], layout={'border': 'solid 1px'})
afc_box = HBox([afc, afc_type], layout={'border': 'solid 1px'})
right_box = VBox([start_b, band_w, afc_box, wdrc_box, play_box, out])
fsliders = VBox([freping_alpha[x] for x in range(cur_bands)])
left_box = VBox([freping, fsliders, freping_reset], layout={'border': 'solid 1px'})
frep_box = HBox([left_box, right_box])
gui = VBox([audio_fig, fft_fig, frep_box])
band_cb('')  # fetch current values
gui